In [2]:
%load_ext autoreload
# %autoreload 2

In [2]:
%autoreload 2
from src import matriz_aumentada, gauss_jordan

A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
b = [1, -2, 3, 4]

Ab = matriz_aumentada(A, b) # type: ignore
gauss_jordan(Ab)

[01-28 23:06:13][INFO][Jonathan Zea] 2026-01-28 23:06:13.470125
[01-28 23:06:13][INFO][Jonathan Zea] 2026-01-28 23:06:13.663661
[01-28 23:06:13][INFO][Jonathan Zea] 2026-01-28 23:06:13.663661
[01-28 23:06:13][INFO][Jonathan Zea] 
[[ 1.  2.  3.  4.  1.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0. -1. -3. -6.  0.]]
[01-28 23:06:13][INFO][Jonathan Zea] 
[[ 1.  0.  3.  6.  9.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0.  0. -3. -7. -4.]]
[01-28 23:06:13][INFO][Jonathan Zea] 
[[ 1.  0.  0. -3.  9.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0.  0.  0.  2. -4.]]
[01-28 23:06:13][INFO][Jonathan Zea] 
[[ 1.  0.  0.  0.  3.]
 [ 0.  1.  0.  0. -6.]
 [ 0.  0. -1.  0. -6.]
 [ 0.  0.  0.  2. -4.]]


array([ 3., -6.,  6., -2.])

# Resolver

In [3]:
import logging
from sys import stdout
from datetime import datetime
import os

logging.basicConfig(
    level=logging.INFO,
    format=f"[%(asctime)s][%(levelname)s][{os.environ.get('USERNAME')}] %(message)s",
    stream=stdout,
    datefmt="%m-%d %H:%M:%S",
)
logging.info(datetime.now())

[02-09 13:24:56][INFO][pc] 2026-02-09 13:24:56.722644


In [4]:
import numpy as np
def matriz_aumentada(A: np.ndarray, I: np.ndarray) -> np.ndarray:
    """Construye la matriz aumentada de un sistema de ecuaciones lineales.

    ## Parameters

    ``A``: matriz de coeficientes.

    ``b``: vector de términos independientes.

    ## Return

    ``Ab``: matriz aumentada.

    """
    n,m = np.shape(A) 
    if not isinstance(A, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        A = np.array(A, dtype=float)
    if not isinstance(I, np.ndarray):
        I = np.array(I, dtype=float)
    assert A.shape[0] == I.shape[0], "Las dimensiones de A y b no coinciden."
    return np.hstack((A, I.reshape(n, m)))

In [5]:
import numpy as np
def separar_m_aumentada(Ab: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Separa la matriz aumentada en la matriz de coeficientes y el vector de términos independientes.

    ## Parameters
    ``Ab``: matriz aumentada.

    ## Return
    ``A``: matriz de coeficientes.
    ``b``: vector de términos independientes.
    """
    n = Ab.shape[0]
    logging.debug(f"Ab = \n{Ab}")
    if not isinstance(Ab, np.ndarray):
        logging.debug("Convirtiendo Ab a numpy array")
        Ab = np.array(Ab, dtype=float)
    return Ab[:, :n], Ab[:, n:]


In [18]:
import numpy as np

def inv_matrix(A: np.ndarray) -> np.ndarray:
    """Resuelve un sistema de ecuaciones lineales mediante el método de Gauss-Jordan.

    ## Parameters

    ``A``: matriz aumentada del sistema de ecuaciones lineales. Debe ser de tamaño n-by-(n+1), donde n es el número de incógnitas.

    ## Return

    ``solucion``: vector con la solución del sistema de ecuaciones lineales.

    """
    if not isinstance(A, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        A = np.array(
            A, dtype=float
        )  # convertir en float, porque si no, convierte en enteros
    n = A.shape[0]

    for i in range(0, n):  # loop por columna

        # --- encontrar pivote
        p = None  # default, first element
        for pi in range(i, n):
            if A[pi, i] == 0:
                # must be nonzero
                continue

            if p is None:
                # first nonzero element
                p = pi
                continue

            if abs(A[pi, i]) < abs(A[p, i]):
                p = pi

        if p is None:
            # no pivot found.
            logging.info(f"\n{A}")
            raise ValueError("No existe solución única.")

        if p != i:
            logging.info(f"Intercambiando filas {i} y {p}.")
            # swap rows
            _aux = A[i, :].copy()
            A[i, :] = A[p, :].copy()
            A[p, :] = _aux
        A[i,:] = A[i,:] / A[i, i]
        # --- Eliminación: loop por fila
        for j in range(0, n):
            if i == j:
                continue
            m = A[j, i] / A[i, i]
            A[j, i:] = A[j, i:] - m * A[i, i:]

        logging.info(f"\n{A}")
            
    return A

## Ejemplos
* Ejemplo 1

In [5]:
# La matriz A =
A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
# tiene como inversa
# A_inv =[[ 0.5, -0.5, -1.5,  1.5],
#        [-0.5,  1.5, -1.5,  0.5],
#        [-1.5, -1.5,  3.5, -1.5],
#        [ 1.5,  0.5, -1.5,  0.5]]
inv_matrix(A)

* Ejemplo 2

In [6]:
# La matriz A =
A = [
    [4, 4, 5, 1],
    [3, 4, 2, 2],
    [2, 1, 4, 1],
    [3, 2, 5, 4],
]
# tiene como inversa
# A_inv =[[-34.,  31.,  52., -20.],
#         [ 19., -17., -29.,  11.],
#         [ 12., -11., -18.,   7.],
#         [  1.,  -1.,  -2.,   1.]]
inv_matrix(A)

## Ejercicios

* Ejercicio 1

In [19]:
A = [[2, -3], [-1, 1]]
Identidad = [[1,0],[0,1]]

matriz=matriz_aumentada(A,Identidad)

inversa= inv_matrix(matriz)
I, A1= separar_m_aumentada(inversa)

print ('\n',A1)


[02-09 15:36:56][INFO][pc] Intercambiando filas 0 y 1.
[02-09 15:36:56][INFO][pc] 
[[ 1. -1. -0. -1.]
 [ 0. -1.  1.  2.]]
[02-09 15:36:56][INFO][pc] 
[[ 1.  0. -1. -3.]
 [-0.  1. -1. -2.]]

 [[-1. -3.]
 [-1. -2.]]


* Ejercicio 2

In [20]:
import numpy as np
A = [
    [4, 0, 0, 5],
    [1, 0, 4, 0],
    [3, 4, 1, 3],
    [1, 3, 3, 0],
]
n = len(A)
Identidad = np.eye(n) 

matriz=matriz_aumentada(A,Identidad)

inversa= inv_matrix(matriz)
I, A1= separar_m_aumentada(inversa)

print ('\n',A1)




[02-09 15:37:05][INFO][pc] Intercambiando filas 0 y 1.
[02-09 15:37:05][INFO][pc] 
[[  1.   0.   4.   0.   0.   1.   0.   0.]
 [  0.   0. -16.   5.   1.  -4.   0.   0.]
 [  0.   4. -11.   3.   0.  -3.   1.   0.]
 [  0.   3.  -1.   0.   0.  -1.   0.   1.]]
[02-09 15:37:05][INFO][pc] Intercambiando filas 1 y 3.
[02-09 15:37:05][INFO][pc] 
[[  1.           0.           4.           0.           0.
    1.           0.           0.        ]
 [  0.           1.          -0.33333333   0.           0.
   -0.33333333   0.           0.33333333]
 [  0.           0.          -9.66666667   3.           0.
   -1.66666667   1.          -1.33333333]
 [  0.           0.         -16.           5.           1.
   -4.           0.           0.        ]]
[02-09 15:37:05][INFO][pc] 
[[ 1.          0.          0.          1.24137931  0.          0.31034483
   0.4137931  -0.55172414]
 [ 0.          1.          0.         -0.10344828  0.         -0.27586207
  -0.03448276  0.37931034]
 [-0.         -0.         

* Ejercicio 3

In [21]:
A = [
    [0., 0., 0., 0., 0., 0., 1., -1.],
    [0., 1., -1., 1., 0., -1., 0., 1.],
    [-1., -1., 0., 0., 2., 1., 0., 0.],
    [-1., -1., -1., 1., 2., 0., 0., 1.],
    [-1., 1., 1., 0., -1., -1., 0., 2.],
    [0., 1., 0., 0., -1., -1., 0., 0.],
    [1., -1., -1., 1., 2., 1., 0., 2.],
    [2., 0., 0., 0., 0., 1., 2., 0.],
]
n = len(A)
Identidad = np.eye(n) 

matriz=matriz_aumentada(A,Identidad)

inversa= inv_matrix(matriz)
I, A1= separar_m_aumentada(inversa)

print ('\n',A1)

[02-09 15:37:13][INFO][pc] Intercambiando filas 0 y 2.
[02-09 15:37:13][INFO][pc] 
[[ 1.  1. -0. -0. -2. -1. -0. -0. -0. -0. -1. -0. -0. -0. -0. -0.]
 [ 0.  1. -1.  1.  0. -1.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1. -1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  1.  0. -1.  0.  1.  0.  0. -1.  1.  0.  0.  0.  0.]
 [ 0.  2.  1.  0. -3. -2.  0.  2.  0.  0. -1.  0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0. -1. -1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0. -2. -1.  1.  4.  2.  0.  2.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 0. -2.  0.  0.  4.  3.  2.  0.  0.  0.  2.  0.  0.  0.  0.  1.]]
[02-09 15:37:13][INFO][pc] 
[[ 1.  0.  1. -1. -2.  0. -0. -1. -0. -1. -1. -0. -0. -0. -0. -0.]
 [ 0.  1. -1.  1.  0. -1.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1. -1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  1.  0. -1.  0.  1.  0.  0. -1.  1.  0.  0.  0.  0.]
 [ 0.  0.  3. -2. -3.  0.  0.  0.  0. -2. -1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1. -1.

* Ejercicio 4

In [22]:
A = [
    [1., 0., 0., 0., -1., 0., 0., -1., 1., -1.],
    [1., 1., 0., -1., -1., 1., 0., 0., 1., -1.],
    [-1., 0., -1., 0., 0., 0., -1., 1., 0., 0.],
    [0., 0., -1., 0., -1., -1., 1., 0., 1., 0.],
    [-1., 0., 0., -1., 1., 1., 1., 1., 0., -1.],
    [1., 0., 0., 1., -1., -1., -1., 1., -1., 0.],
    [1., 1., 1., 0., 1., 0., -1., -1., -1., 1.],
    [1., 1., 1., 1., 0., 0., 1., 1., 0., 0.],
    [1., 1., 1., 1., 1., 0., -1., -1., 0., 0.],
    [0., 0., -1., -1., -1., 0., 1., 1., 1., -1.],
]
n = len(A)
Identidad = np.eye(n) 

matriz=matriz_aumentada(A,Identidad)

inversa= inv_matrix(matriz)
I, A1= separar_m_aumentada(inversa)

print ('inversa\n',A1)

[02-09 15:37:23][INFO][pc] 
[[ 1.  0.  0.  0. -1.  0.  0. -1.  1. -1.  1.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  1.  0. -1.  0.  1.  0.  1.  0.  0. -1.  1.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0. -1.  0. -1.  0. -1.  0.  1. -1.  1.  0.  1.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0. -1.  0. -1. -1.  1.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0. -1.  0.  1.  1.  0.  1. -2.  1.  0.  0.  0.  1.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1.  0. -1. -1.  2. -2.  1. -1.  0.  0.  0.  0.  1.  0.  0.
   0.  0.]
 [ 0.  1.  1.  0.  2.  0. -1.  0. -2.  2. -1.  0.  0.  0.  0.  0.  1.  0.
   0.  0.]
 [ 0.  1.  1.  1.  1.  0.  1.  2. -1.  1. -1.  0.  0.  0.  0.  0.  0.  1.
   0.  0.]
 [ 0.  1.  1.  1.  2.  0. -1.  0. -1.  1. -1.  0.  0.  0.  0.  0.  0.  0.
   1.  0.]
 [ 0.  0. -1. -1. -1.  0.  1.  1.  1. -1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  1.]]
[02-09 15:37:23][INFO][pc] 
[[ 1.  0.  0.  0. -1.  0.  0. -1.  1. -1.  1.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  1